In [1]:
from keras.layers import Dense,Conv2D,MaxPooling2D,UpSampling2D
from keras import Input, Model
from keras.datasets import mnist
import wfdb
from wfdb import processing
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from sklearn.decomposition import PCA
import tensorflow as tf
tf.config.run_functions_eagerly(True)
%matplotlib inline

In [6]:
record_number = '05091'
filepath = 'D:/phd/mit-bih-atrial-fibrillation-database-1.0.0/files/' + record_number
record = wfdb.rdrecord(filepath)#, sampto = 10000) 
annotation = wfdb.rdann(filepath,'atr', shift_samps=True)#, sampto = 10000)

In [7]:
sig = record.p_signal[:,0]
sig_peaks, _ = signal.find_peaks(sig)
t = []
for i in sig_peaks:
    if sig[i] > 0.25:
        t.append(i)
s = []


for i in t:
    r = wfdb.rdrecord(filepath, sampfrom=i-100, sampto=i+100)
    s.append(r.p_signal[:,0])

In [8]:
encoding_dim = 64 
input_img = Input(shape=(200,))
# encoded representation of input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# decoded representation of code 
decoded = Dense(200, activation='sigmoid')(encoded)
# Model which take input image and shows decoded images
autoencoder = Model(input_img, decoded)

In [9]:
# This model shows encoded images
encoder = Model(input_img, encoded)
# Creating a decoder model
encoded_input = Input(shape=(encoding_dim,))
# last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [10]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [11]:
m = np.array(s)
autoencoder.fit(m, m, epochs=15, batch_size=256)

C:\Users\houfo\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/15
1207/1207 [==============================] - 54s 44ms/step - loss: -720.6014
Epoch 2/15
1207/1207 [==============================] - 57s 47ms/step - loss: -4725.4995
Epoch 3/15
1207/1207 [==============================] - 57s 47ms/step - loss: -11820.2080
Epoch 4/15
1207/1207 [==============================] - 53s 44ms/step - loss: -21577.5430
Epoch 5/15
1207/1207 [==============================] - 52s 43ms/step - loss: -33630.7969
Epoch 6/15
1207/1207 [==============================] - 54s 44ms/step - loss: -47803.8359
Epoch 7/15
1207/1207 [==============================] - 50s 41ms/step - loss: -64118.3945
Epoch 8/15
1207/1207 [==============================] - 49s 41ms/step - loss: -82527.7344
Epoch 9/15
1207/1207 [==============================] - 56s 47ms/step - loss: -102917.3672
Epoch 10/15
1207/1207 [==============================] - 50s 41ms/step - loss: -125294.5547
Epoch 11/15
1207/1207 [==============================] - 48s 40ms/step - loss: -149494.6562
Epoch 12

In [4]:
def process_row(record_number):
    filepath = 'D:/phd/mit-bih-atrial-fibrillation-database-1.0.0/files/' + record_number
    record = wfdb.rdrecord(filepath)#, sampto = 10000)
    ann = wfdb.rdann(filepath,'atr')
    sig = record.p_signal[:,0]
    sig_peaks, _ = processing.find_peaks(sig)
    t = []
    for i in sig_peaks:
        if sig[i] > 0.25:
            t.append(i)
    s = []
    annIndexes = []
    for i in range(len(sig_peaks)):
        val = sig_peaks[i]
        if val < 100 or val+100 > len(sig):
            continue
        r = wfdb.rdrecord(filepath, sampfrom=val-100, sampto=val+100)
        for j in ann.sample:
            if val-100 < j and val + 100 > j:
                annIndexes.append(i)
                s.append(r.p_signal[:,0])
                break
        
    
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(s)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
    principalDf["color"] = "blue"
    #principalDf.loc[annIndexes, "color"] = "red"
    addInfo = ""
    if len(annIndexes) == 0:
        addInfo = "empty"
    principalDf.plot(x ='principal component 1', y='principal component 2', alpha=0.5, kind = 'scatter', title = "Record = "+record_number + " " + addInfo, color=principalDf["color"])

In [ ]:
a = [
'04015',
'04043',
'04048',
'04126',
'04746',
'04908',
'04936',
'05091',
'05121',
'05261',
'06426',
'06453',
'06995',
'07162',
'07859',
'07879',
'07910',
'08215',
'08219',
'08378',
'08405',
'08434',
'08455',
]
for row in a:
    print(row)
    process_row(row)
        

04015
04043
04048
04126


In [ ]:
# Example below can be useful in the feature, for result validation

In [ ]:
# Use xqrs detection algorithm and compare against reference annotations
sig, fields = wfdb.rdsamp('../input/bidmc-congestive-heart-failure/files/chf03', channels=[0], sampto=15000)
ann_ref = wfdb.rdann('../input/bidmc-congestive-heart-failure/files/chf03','ecg', sampto=15000)

# Run qrs detection on signal
xqrs = processing.XQRS(sig=sig[:,0], fs=fields['fs'])
xqrs.detect()
# Alternatively, use the gateway function to get the qrs indices directly
# qrs_inds = processing.xqrs_detect(sig=sig[:,0], fs=fields['fs'])

# Compare detected qrs complexes to reference annotation.
# Note, first sample in 100.atr is not a qrs.
comparitor = processing.compare_annotations(ref_sample=ann_ref.sample[1:],
                                            test_sample=xqrs.qrs_inds,
                                            window_width=int(0.1 * fields['fs']),
                                            signal=sig[:,0])


In [18]:
p = np.array(s[25])
autoencoder.predict(p.reshape(1,-1))

1/1 [==============================] - 0s 28ms/step


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [15]:
p.reshape(1,-1)

array([[ 0.005,  0.03 ,  0.005,  0.005, -0.015, -0.035, -0.025, -0.01 ,
        -0.005, -0.01 ,  0.01 ,  0.015, -0.01 , -0.01 , -0.025, -0.035,
        -0.03 , -0.015, -0.03 , -0.04 ,  0.   ,  0.005,  0.005,  0.   ,
         0.015, -0.02 , -0.025, -0.035, -0.015, -0.02 , -0.02 , -0.02 ,
        -0.01 ,  0.005,  0.005, -0.005, -0.005, -0.005, -0.025, -0.02 ,
        -0.01 , -0.02 ,  0.   ,  0.015,  0.015,  0.035,  0.03 ,  0.03 ,
         0.05 ,  0.01 , -0.01 ,  0.   ,  0.02 ,  0.065,  0.07 ,  0.095,
         0.09 ,  0.15 ,  0.15 ,  0.135,  0.125,  0.125,  0.14 ,  0.15 ,
         0.145,  0.155,  0.135,  0.135,  0.1  ,  0.055,  0.04 ,  0.045,
         0.025,  0.02 ,  0.02 ,  0.   ,  0.01 ,  0.02 ,  0.   ,  0.01 ,
         0.   ,  0.   , -0.005, -0.035, -0.04 , -0.02 , -0.005,  0.   ,
         0.   ,  0.03 ,  0.02 ,  0.025,  0.015,  0.005,  0.065,  0.16 ,
         0.25 ,  0.33 ,  0.49 ,  0.68 ,  0.775,  0.675,  0.445,  0.03 ,
        -0.425, -0.815, -1.11 , -1.15 , -0.905, -0.66 , -0.49 , 